In [1]:
import numpy as np
# from tolsolvty import tolsolvty

In [2]:
def write_matrix_as_intervals(filename, A):
  (m, n) = A.shape
  with open(filename, "a") as mtxfile:
    for i in range(m):
      for j in range(n):
        mtxfile.write(str(A[i][j]) + "  " + str(A[i][j]) + "  ")
      mtxfile.write("\n")
    mtxfile.write("\n")


def write_vector_as_intervals(filename, infb, supb):
  with open(filename, "a") as vfile:
    for i in range(len(infb)):
      vfile.write(str(infb[i]) + "  " + str(supb[i]) + "  ")


def write_header(filename, size):
  with open(filename, "w") as mtxfile:
    mtxfile.write(str(size) + " 1.E-9      1.     100")
    mtxfile.write("\n\n")

In [3]:
def add_ans_for_bad_matrix(filename):
  a_file = open(filename, "r")
  lines = a_file.readlines()
  a_file.close()
  with open(filename, "w") as new_file:
    dim = 0
    for line in lines:
      if "Dimension" in line:
        dim = int(line.split('=')[-1])
      elif "matrix" in line:
        for _ in range(dim):
          new_file.write('-1000000000.0, 1000000000.0\n')
      else:
        new_file.write(line)

In [4]:
def clear_columns(M):
  leftIndxs = np.arange(18)
  idx = np.argwhere(np.all(M[..., :] == 0.0, axis=0))
  M = np.delete(M, idx, axis=1)
  ind = 18
  ind = ind - len(idx)
  leftIndxs = np.delete(leftIndxs, idx)
  M = M[:ind]
  while len(idx) > 0:
    idx = np.argwhere(np.all(M[..., :] == 0.0, axis=0))
    M = np.delete(M, idx, axis=1)
    ind = ind - len(idx)
    leftIndxs = np.delete(leftIndxs, idx)
    M = M[:ind]
  return M, ind, leftIndxs


def clear_lines(M):
  return M[~(M==0).all(1)]

In [5]:
def read_inters_on_ind(indSolved, coords, filename, iteration):
  with open(filename, "r") as read_file:
    for ind_arr in indSolved[iteration]:
      for i in ind_arr:
        interval = read_file.readline().split(',')
        coords[iteration][i] += [np.array(interval).astype(np.double)]

In [6]:
def intersect(a_, b_):
  if a_ is None:
    return None
  if b_ is None:
    return None 
  if a_[0] > a_[1]:
    buf = a_[1]
    a_[1] = a_[0]
    a_[0] = buf
  if b_[0] > b_[1]:
    buf = b_[1]
    b_[1] = b_[0]
    b_[0] = buf 
  if a_[1] < b_[0] or b_[1] < a_[0]:
    return None
  return np.array([max(a_[0], b_[0]), min(a_[1], b_[1])])

In [14]:
path_folder = ''
M1 = np.loadtxt(path_folder + 'matrix_n_phi_1.txt')
M6 = np.loadtxt(path_folder + 'matrix_n_phi_6.txt')

In [15]:
M1_lu = M1[0:128,0:18]
M1_ld = M1[128:, 0:18]
M1_ru = M1[0:128, 18:]
M1_rd = M1[128:, 18:]

In [16]:
M_128_18 = [clear_lines(M1_ld), clear_lines(M1_ru)]
M_128_18[1].shape

(128, 18)

In [17]:
xs = np.array([])
coords = [dict(zip(range(18), [[] for _ in range(18)])), dict(zip(range(18), [[] for _ in range(18)]))]
indSolved = {0:[], 1:[]}
for mInd in range(len(M_128_18)):
  M = M_128_18[mInd]
  lines = M.shape[0]
  idxs = np.argsort([np.count_nonzero(M[i]==0) for i in range(len(M))])
  x = np.array(np.random.randint(1, 10, 18))
  r = np.random.randint(1, 5, lines)
  xs = np.append(xs, x)
  b = M_128_18[0].dot(x)
  infb = b - r
  supb = b + r
  begin = 0
  mtx_num = 0
  while begin < 128:
    M_work, add, leftInd = clear_columns(M[idxs[begin:begin+18]])
    if add == 1:
      coords[mInd][leftInd[0]] += [np.array([infb[idxs[begin]], supb[idxs[begin]]])]
    else:
      indSolved[mInd] += [leftInd]
      s_mtx_num = str(mtx_num) if mtx_num >= 10 else '0' + str(mtx_num)
      filename = 'matrix' + s_mtx_num + '_' + str(mInd) + '.txt'
      # write_header(filename, add)
      if M_work.shape[0] != M_work.shape[1]:
        print("ALERT!")
      # write_matrix_as_intervals(filename, M_work)
      # write_vector_as_intervals(filename, infb[idxs[begin:begin+add]], supb[idxs[begin:begin+add]])
    begin += add
    mtx_num += 1
    #_x[i] = solve(M_work, infb[idxs[i:i+18]], supb[idxs[i:i+18]]) 

In [18]:
add_ans_for_bad_matrix("solutions0.txt")
read_inters_on_ind(indSolved, coords, "solutions0.txt", 0)
ans_inter = []
for i in range(18):
  coord = coords[0][i]
  res_inter = np.array([-float('infinity'), float('infinity')])
  for inter in coord:
    res_inter = intersect(res_inter, inter)
  ans_inter += [res_inter]

In [19]:
with open("answer0.txt", "w") as ansFile:
  ansFile.write(' '.join(map(str, ans_inter)))

In [20]:
add_ans_for_bad_matrix("solutions1.txt")
read_inters_on_ind(indSolved, coords, "solutions1.txt", 1)
ans_inter = []
for i in range(18):
  coord = coords[1][i]
  res_inter = np.array([-float('infinity'), float('infinity')])
  for inter in coord:
    res_inter = intersect(res_inter, inter)
  ans_inter += [res_inter]

In [21]:
with open("answer1.txt", "w") as ansFile:
  ansFile.write(' '.join(map(str, ans_inter)))

In [ ]:
coords[0]

In [32]:
print(xs)

[8. 7. 5. 5. 1. 9. 6. 5. 9. 2. 2. 5. 5. 1. 8. 8. 2. 7. 7. 1. 6. 5. 1. 6.
 9. 7. 5. 1. 6. 4. 2. 5. 6. 3. 2. 8.]
